In [1]:
!pip install pandas
!pip install git+https://github.com/descriptinc/audiotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 70.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 73.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-req-build-ofdhjs49
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-req-build-ofdhjs49
  Resolved https://github.com/descriptinc/audiotools to commit 784f87b7679daee82ee7bb25770d5bd51a977bb6
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.5 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 12.9 

In [2]:
import os
import pandas as pd
from audiotools import AudioSignal  # audiotools 모듈에서 AudioSignal 클래스를 불러오기


In [10]:
root_dir = "./data/"  # 오디오 파일이 있는 루트 디렉토리
save_to='audio_info.csv'
batch_size=100
# 이미 처리된 audio_path들을 읽어와 집합으로 만들기
if os.path.exists(save_to):
    existing_df = pd.read_csv(save_to)
    processed_paths = set(existing_df['audio_path'])
else:
    processed_paths = set()

# 오디오 파일 탐색
audio_files = [os.path.join(root, f) for root, dirs, files in os.walk(root_dir)
               for f in files if f.endswith(('.wav', '.mp3')) and os.path.join(root, f) not in processed_paths]
print("Dataset size: ", len(audio_files))

dataset size:  243137


In [11]:
data = []  # 오디오 정보를 저장할 리스트

for idx, audio_path in enumerate(audio_files):
    try:
        audio = AudioSignal(audio_path)
        duration = audio.signal_duration  # 오디오 지속 시간 추출
        data.append({'audio_path': audio_path, 'duration': duration})
        
        # 100개의 오디오마다 혹은 마지막 오디오 파일이면 CSV 파일 업데이트
        if (idx + 1) % batch_size == 0 or idx + 1 == len(audio_files):
            df = pd.DataFrame(data)
            df.to_csv(save_to, index=False, mode='a', header=not os.path.exists(save_to))
            data = []  # 리스트 초기화
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")

Error processing ./data/bbc-sfx-audios/Daily_Life/NHU9249427.wav: Audio file ./data/bbc-sfx-audios/Daily_Life/NHU9249427.wav with offset 0 and duration None is empty!


In [12]:
import pandas as pd

# audio_info.csv 파일을 DataFrame으로 로드
df = pd.read_csv('audio_info.csv')

# duration이 3초 이하인 행만 필터링
filtered_df = df[df['duration'] <= 3]

# 결과를 새 파일로 저장
filtered_df.to_csv('filtered_audio_info.csv', index=False)


In [ ]:
def process_audio_files(root_dir, save_to='audio_info.csv', batch_size=100):
    # 이미 처리된 audio_path들을 읽어와 집합으로 만들기
    if os.path.exists(save_to):
        existing_df = pd.read_csv(save_to)
        processed_paths = set(existing_df['audio_path'])
    else:
        processed_paths = set()

    # 오디오 파일 탐색
    audio_files = [os.path.join(root, f) for root, dirs, files in os.walk(root_dir)
                   for f in files if f.endswith(('.wav', '.mp3')) and os.path.join(root, f) not in processed_paths]
    data = []  # 오디오 정보를 저장할 리스트

    for idx, audio_path in enumerate(audio_files):
        try:
            audio = AudioSignal(audio_path)
            duration = audio.signal_duration  # 오디오 지속 시간 추출
            data.append({'audio_path': audio_path, 'duration': duration})
            
            # 100개의 오디오마다 혹은 마지막 오디오 파일이면 CSV 파일 업데이트
            if (idx + 1) % batch_size == 0 or idx + 1 == len(audio_files):
                df = pd.DataFrame(data)
                df.to_csv(save_to, index=False, mode='a', header=not os.path.exists(save_to))
                data = []  # 리스트 초기화
        except Exception as e:
            print(f"Error processing {audio_path}: {e}")

# 함수 사용 예시
#root_dir = "./data/"  # 오디오 파일이 있는 루트 디렉토리
#process_audio_files(root_dir)
